In [40]:
import pandas as pd

In [41]:
from sklearn.tree import DecisionTreeClassifier

In [42]:
df = pd.read_csv('id3.csv - id3.csv.csv')

In [43]:
df.head()

,Outlook,Temperature,Humidity,Wind,Answer
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes


In [44]:
df

,Outlook,Temperature,Humidity,Wind,Answer
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rain,mild,normal,weak,yes


In [6]:
import math
import csv

In [7]:
def load_csv(filename):
    lines = csv.reader(open(filename,"r"))
    dataset = list(lines)
    headers = dataset.pop(0)
    return dataset,headers

In [33]:
class Node:
    def __init__(self,attribute):
        self.attribute=attribute
        self.children = []
        self.answer = ""

In [34]:
def subtables(data,col,delete):
    dic = {}
    coldata = [row[col] for row in data]
    attr = list(set(coldata))
    
    counts=[0]*len(attr)
    r = len(data)
    c = len(data[0])
    for x in range(len(attr)):
        for y in range(r):
            if data[y][col]==attr[x]:
                counts[x]+=1
    for x in range(len(attr)):
        dic[attr[x]]=[[0 for i in range(c)] for j in range(counts[x])]
        pos = 0
        for y in range(r):
            if data[y][col]==attr[x]:
                if delete:
                    del data[y][col]
                dic[attr[x]][pos]=data[y]
                pos+=1
    return attr,dic

In [35]:
import math

def entropy(S):
    attr = list(set(S))
    
    if len(attr) == 1:
        return 0  # Return 0 entropy when there's only one attribute
    
    counts = [0] * len(attr)
    
    for i in range(len(attr)):
        counts[i] = sum(1 for x in S if attr[i] == x) / (len(S) * 1.0)
    
    sums = 0
    for cnt in counts:
        sums += -1 * cnt * math.log(cnt, 2)
    
    return sums

def compute_gain(data, col):
    attr, dic = subtables(data, col, delete=False)
    total_size = len(data)
    entropies = [0] * len(attr)
    ratio = [0] * len(attr)
    total_entropy = entropy([row[-1] for row in data])
    
    for x in range(len(attr)):
        ratio[x] = len(dic[attr[x]]) / (total_size * 1.0)
        entropies[x] = entropy([row[-1] for row in dic[attr[x]]])
        total_entropy -= ratio[x] * entropies[x]
    
    return total_entropy




In [36]:
def build_tree(data, features):
    lastcol = [row[-1] for row in data]

    if len(set(lastcol)) == 1:
        node = Node("")  # Assuming you have a Node class defined
        node.answer = lastcol[0]
        return node

    n = len(data[0]) - 1
    gains = [0] * n

    for col in range(n):
        gains[col] = compute_gain(data, col)

    split = gains.index(max(gains))

    node = Node(features[split])  # Assuming you have a Node class defined
    fea = features[:split] + features[split + 1:]

    attr, dic = subtables(data, split, delete=True)  # Assuming you have implemented subtables function

    for x in range(len(attr)):
        child = build_tree(dic[attr[x]], fea)  # Corrected the assignment operator from '-' to '='
        node.children.append((attr[x], child))

    return node



In [37]:
def print_tree(node, level):
    if node.answer != "":
        print(" " * level, node.answer)
        return

    print(" " * level, node.attribute)

    for value, n in node.children:
        print(" " * (level + 1), value)
        print_tree(n, level + 2)



In [38]:
def classify(node, x_test, features):
    if node.answer != "":
        print(node.answer)
        return

    pos = features.index(node.attribute)

    for value, n in node.children:
        if x_test[pos] == value:
            classify(n, x_test, features)

In [45]:
dataset, features = load_csv("id3.csv - id3.csv.csv")
node1 = build_tree(dataset, features)

print("The decision tree for the dataset using ID3 algorithm is")
print_tree(node1, 8)

testdata, features = load_csv("id3_test.csv - id3_test.csv.csv")

for xtest in testdata:
    print("The test instance:", xtest)
    print("The label for test instance:", end="")
    classify(node1, xtest, features)

The decision tree for the dataset using ID3 algorithm is
         Outlook
          overcast
           yes
          sunny
           Humidity
            high
             no
            normal
             yes
          rain
           Wind
            weak
             yes
            strong
             no
The test instance: ['rain', 'cool', 'normal', 'strong']
The label for test instance:no
The test instance: ['sunny', 'mild', 'normal', 'strong']
The label for test instance:yes
The test instance: ['overcast', 'mild', 'high', 'strong']
The label for test instance:yes
